In [ ]:
# colab users (only): install warpfield with pip
!pip -q install warpfield

In [ ]:
# download some example data (see https://github.com/andreasmang/nirep)
!wget -nv https://github.com/andreasmang/nirep/raw/refs/heads/master/nifti/na01.nii.gz
!wget -nv https://github.com/andreasmang/nirep/raw/refs/heads/master/nifti/na15.nii.gz

In [ ]:
import warpfield

In [ ]:
# load data
fixed, _ = warpfield.load_data("na01.nii.gz")
moving, _ = warpfield.load_data("na15.nii.gz")

In [ ]:
# inspect data
from matplotlib import pyplot as plt
plt.figure()
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(fixed.mean(0), cmap="gray")
ax[0].set_title("fixed")
ax[1].imshow(moving.mean(0), cmap="gray")
ax[1].set_title("moving")

In [ ]:
# create a basic recipe with one affine registration level and a default pre-filter:
recipe = warpfield.Recipe()
recipe.pre_filter.clip_thresh = 0  # clip DC background, if present

# affine level properties
recipe.levels[0].repeats = 10
recipe.levels[0].update_rate = 0.8  # lowered to dampen oscillations

# add non-rigid registration levels:
recipe.add_level(block_size=[128, 128, 128])
recipe.levels[-1].smooth.sigmas = [1.0, 1.0, 1.0]
recipe.levels[-1].repeats = 5

recipe.add_level(block_size=[64, 64, 64])
recipe.levels[-1].block_stride = 0.5
recipe.levels[-1].smooth.sigmas = [1.0, 1.0, 1.0]
recipe.levels[-1].repeats = 5

recipe.add_level(block_size=[32, 32, 32])
recipe.levels[-1].block_stride = 0.5
recipe.levels[-1].smooth.sigmas = [2.0, 2.0, 2.0]
recipe.levels[-1].repeats = 5

In [ ]:
# register (without video)
moving_reg, warpmap, _ = warpfield.register_volumes(fixed, moving, recipe)

In [ ]:
# register (with MIP video).
video_path = "output.mp4"
units_per_voxel = [1,1,1]
callback = warpfield.utils.mips_callback(units_per_voxel=units_per_voxel)
moving_reg, warpmap, _ = warpfield.register_volumes(fixed, moving, recipe, video_path=video_path, callback=callback)
warpfield.utils.showvid(video_path, embed=True)

In [ ]:
# register (with slice mosaic video)
video_path = "output.mp4"
units_per_voxel = [1, 1, 1]
callback = warpfield.utils.mosaic_callback(units_per_voxel=units_per_voxel, axis=0, num_slices=9)
moving_reg, warpmap, _ = warpfield.register_volumes(fixed, moving, recipe, video_path=video_path, callback=callback)
warpfield.utils.showvid(video_path, embed=True)